## Merging Dataframes
Bringing multiple dataframes together by joining horizontally or concatenating vertically

It is well to be able to translate some concepts, terminology and understand equivalency across disciplines (database vs mathematical)
 - __Relational Theory : Set Theory__
 - Full Outer Join : Union
 - Inner Join : Intersection

In [9]:
import pandas as pd
# set up two data sets back patients and hip patients
# can do this as a list of dicts
back_df = pd.DataFrame([{'MRN': '12345', 'dx': 'severe back pain'},
                        {'MRN': '23456', 'dx': 'moderate back pain'},
                        {'MRN': '34567', 'dx': 'mild back pain'}])
# index by MRN
back_df = back_df.set_index('MRN')
hip_df = pd.DataFrame([{'MRN': '12345', 'dx': 'right hip pain'},
                       {'MRN': '45678', 'dx': 'dislocation'},
                       {'MRN': '34567', 'dx': 'fracture'}])
hip_df = hip_df.set_index('MRN')
back_df
hip_df

,dx
MRN,
12345,right hip pain
45678,dislocation
34567,fracture


## merging the datasets -- Union (outer join)

In [10]:
pd.merge (back_df, hip_df, how='outer', left_index=True, right_index=True)


,dx_x,dx_y
MRN,,
12345,severe back pain,right hip pain
23456,moderate back pain,NaN
34567,mild back pain,fracture
45678,NaN,dislocation


## merging the datasets -- Intersection (inner join)

In [13]:
pd.merge (back_df, hip_df, how='inner', left_index=True, right_index=True)

,dx_x,dx_y
MRN,,
12345,severe back pain,right hip pain
34567,mild back pain,fracture


## merging the datasets -- Set Addition(left join and right joins)


In [21]:
pd.merge(back_df, hip_df, how = 'left', left_index=True, right_index=True)

,dx_x,dx_y
MRN,,
12345,severe back pain,right hip pain
23456,moderate back pain,NaN
34567,mild back pain,fracture


In [19]:
pd.merge(back_df, hip_df, how = 'right', left_index=True, right_index=True)

,dx_x,dx_y
MRN,,
12345,severe back pain,right hip pain
45678,NaN,dislocation
34567,mild back pain,fracture


## Declaring a join without indices, 'on'

In [24]:
# to demonstrate this I'll first unset the indexes added earlier
back_df = back_df.reset_index()
hip_df = hip_df.reset_index()

# then specify the column to join on
pd.merge(back_df, hip_df, how='right', on='MRN')

,level_0,index_x,MRN,dx_x,index_y,dx_y
0,0.0,0.0,12345,severe back pain,0,right hip pain
1,NaN,NaN,45678,NaN,1,dislocation
2,2.0,2.0,34567,mild back pain,2,fracture


## merge conflicts
In the examples above, many of the dx columns had different values from one dataframe to the other. This is a merge conflict. The way Pandas handles that is to split the dx column into two sub columns

dx_x is data from the dataframe on the left, dx_y is data from the dataframe on the right

## multi-column joins
sometimes it will take more than one column to create a unique index in a dataframe. This can be accomodated by multi-column joins which use a list of column names in the "on" parameter of the merge


In [32]:
# revising our dataset to create unique multi-column values
back_df = pd.DataFrame([{'MRN': '12345', 'dept':'spine', 'dx': 'severe back pain'},
                        {'MRN': '23456', 'dept':'spine', 'dx': 'moderate back pain'},
                        {'MRN': '34567', 'dept':'spine', 'dx': 'mild back pain'}])

hip_df = pd.DataFrame([{'MRN': '12345', 'dept':'ortho', 'dx': 'right hip pain'},
                       {'MRN': '45678', 'dept':'ortho', 'dx': 'dislocation'},
                       {'MRN': '34567', 'dept':'spine', 'dx': 'fracture'}])

pd.merge(back_df, hip_df, how="inner", on=['MRN','dept'])
# will produce the records where the combination of values occurs in both data sets

,MRN,dept,dx_x,dx_y
0,34567,spine,mild back pain,fracture


# Concatenating
Merging is joining dataframes horizontally, whereas concatenating is joining dataframes vertically

Generally the columns will have the same names, and the rows will be appended to the bottom.


In [39]:
print (len(back_df))
print (len(hip_df))

combined = pd.concat([back_df, hip_df])
combined


3
3


,MRN,dept,dx
0,12345,spine,severe back pain
1,23456,spine,moderate back pain
2,34567,spine,mild back pain
0,12345,ortho,right hip pain
1,45678,ortho,dislocation
2,34567,spine,fracture


In [37]:
print (len(combined))


6


In [40]:
# if you need to retain the origin of the dataset you can do this by passing in a keys parameter
# this parameter is a list with the same number of entries as the list of dataframes to concatenate
combined = pd.concat([back_df, hip_df], keys=['cdw','pop'])


MRN   dept                  dx
cdw 0  12345  spine    severe back pain
    1  23456  spine  moderate back pain
    2  34567  spine      mild back pain
pop 0  12345  ortho      right hip pain
    1  45678  ortho         dislocation
    2  34567  spine            fracture

In [56]:
# if the columns are not identical between the data set, you can add the join parameter
# with values of inner or outer
# Inner, analagous to left join will only include columns that occur in both data frames. 
# Outer will include all columns in all dataframes with the values not in the other dataframe being conferted to NaN
back_df = pd.DataFrame([{'MRN': '12345', 'dr':'Dr A', 'dx': 'severe back pain'},
                        {'MRN': '23456', 'dr':'Dr B', 'dx': 'moderate back pain'},
                        {'MRN': '34567', 'dr':'Dr C', 'dx': 'mild back pain'}])

hip_df = pd.DataFrame([{'MRN': '12345', 'dept':'ortho', 'dx': 'right hip pain'},
                       {'MRN': '45678', 'dept':'ortho', 'dx': 'dislocation'},
                       {'MRN': '34567', 'dept':'spine', 'dx': 'fracture'}])

combined = pd.concat([back_df, hip_df],keys=['back','hip'], join='inner')
combined

MRN                  dx
back 0  12345    severe back pain
     1  23456  moderate back pain
     2  34567      mild back pain
hip  0  12345      right hip pain
     1  45678         dislocation
     2  34567            fracture

In [57]:
combined = pd.concat([back_df, hip_df],keys=['back','hip'], join='outer')
combined

MRN    dr                  dx   dept
back 0  12345  Dr A    severe back pain    NaN
     1  23456  Dr B  moderate back pain    NaN
     2  34567  Dr C      mild back pain    NaN
hip  0  12345   NaN      right hip pain  ortho
     1  45678   NaN         dislocation  ortho
     2  34567   NaN            fracture  spine